In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Initialize undetected Chrome
driver = uc.Chrome()

def scroll_to_bottom():
    """Scrolls to the bottom of the page to load more hotels."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Wait for new hotels to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:  # No new content loaded
            break
        last_height = new_height

def scrape_booking(city, checkin_date, checkout_date):
    """Scrapes hotel details from Booking.com for given city & dates."""
    search_url = f"https://www.booking.com/searchresults.html?ss={city}&checkin={checkin_date}&checkout={checkout_date}"
    
    driver.get(search_url)
    time.sleep(6)  # Wait for initial page load
    
    # input("Press Enter after solving CAPTCHA (if any)...")  # Handle manual CAPTCHA

    print("Scrolling to load all hotels...")
    scroll_to_bottom()  # Call the scroll function

    hotels = []
    
    hotel_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='property-card']")
    print(f"Total Hotels Found: {len(hotel_elements)}")  # Debugging count

    for hotel in hotel_elements:
        try:
            name = hotel.find_element(By.CSS_SELECTOR, "div[data-testid='title']").text
        except:
            name = "N/A"

        try:
            price = hotel.find_element(By.CSS_SELECTOR, "span[data-testid='price-and-discounted-price']").text
        except:
            price = "N/A"

        try:
            rating = hotel.find_element(By.CSS_SELECTOR, "div[data-testid='review-score']").text
        except:
            rating = "N/A"

        try:
            address = hotel.find_element(By.CSS_SELECTOR, "span[data-testid='address']").text
        except:
            address = "N/A"

        hotels.append({
            "Name": name,
            "Price": price,
            "Rating": rating,
            "Address": address
        })

    # Save to CSV
    if hotels:
        df = pd.DataFrame(hotels)
        df.to_csv("booking_hotels.csv", index=False)
        print("✅ Scraping Complete! Data saved as booking_hotels.csv")
    else:
        print("❌ No data found!")

# Example: Scraping Mumbai hotels for 10th March 2025 - 15th March 2025
scrape_booking("Mumbai", "2025-03-10", "2025-03-15")

# Close the browser
driver.quit()

 

Scrolling to load all hotels...
Total Hotels Found: 75
✅ Scraping Complete! Data saved as booking_hotels.csv
